##### IMPORTAR AS PLANILHAS

In [ ]:
import pandas as pd
from resolve_path import ajuste_path

##### IMPORTANDO PLANILHA

In [ ]:
path_iw = "data/util/os/"

path_iw = ajuste_path(path_iw)

df_iw47 = pd.read_csv(path_iw + "IW47_Executadas_preparado.csv")

df_iw47.columns

In [ ]:
colunas_tipo_atividade = ["Txt.breve operação", "Denominação TAM"]

In [ ]:
df_iw47["Data fim"] = pd.to_datetime(df_iw47["Data fim"])
df_iw47["Data inicio"] = pd.to_datetime(df_iw47["Data inicio"])
df_iw47["Local de instalação"] = df_iw47["Local de instalação"].astype(str)

##### CRIANDO DATAFRAME SEM INCLUIR DATAS FINAIS ANTERIORES A DATAS INICIAIS

In [ ]:
# Removendo linhas onde a data final é superior a data inicial
df_sem_linhas_irregulares = df_iw47.loc[df_iw47["Data fim"]
                                        >= df_iw47["Data inicio"]]

num_linhas_removidas = df_iw47.shape[0] - df_sem_linhas_irregulares.shape[0]
print("Linhas removidas:", num_linhas_removidas)

##### FUNÇÃO QUE ADD COLUNA DE HH POR MES E QUANTIDADE DE MESES PASSADOS DURANTE A OS

In [ ]:
# Soma dos meses, em funcao da quantidade de anos que se passaram e meses, entre o inicio e fim da OS
df = df_sem_linhas_irregulares

df["Meses"] = (
    (df["Data fim"].dt.year - df["Data inicio"].dt.year) * 12
    + (df["Data fim"].dt.month - df["Data inicio"].dt.month)
) + 1
df["HH por mes"] = df["HH final"] / df["Meses"]

##### FUNCAO QUE AGRUPA HH POR ANO, MES, LOCAL E TIPO DE ATIVIDADE

In [ ]:
df["Data inicio"] = df["Data inicio"].apply(lambda x: x.replace(day=1))
df["Data fim"] = df["Data fim"].apply(lambda x: x.replace(day=1))

In [ ]:
# fazer um produto cartesiano entre a IW47 e o espaço de tempo para não precisar de loop

espaco_de_tempo = pd.date_range(
    start='2020-01-01', end=pd.Timestamp.now(), freq="MS")

espaco_de_tempo = pd.DataFrame(
    {'merge': [1]*len(espaco_de_tempo), 'Ano Mes': espaco_de_tempo})


# só expande as linhas onde a operação dura mais de um mes
df.loc[df['Meses'] > 1, 'dura mais de mes'] = 1

# joga fora colunas desnecessárias para salvar memória
colunas_a_manter = [
    'Local de instalação',
    'HH por mes',
    'Meses',
    'dura mais de mes',
    'Data inicio',
    'Data fim'
]
colunas_a_manter.extend(colunas_tipo_atividade)
df = df[colunas_a_manter]

df_expandido = pd.merge(df, espaco_de_tempo,
                        left_on='dura mais de mes',
                        right_on='merge',
                        how='left'
                        ).drop(columns=['merge'])

# trata das colunas de data para que sejam incluidas apropriadamente
df_expandido['dura mais de mes'] = df_expandido['dura mais de mes'].fillna(0)
df_expandido.loc[df_expandido['dura mais de mes']
                 == 0, 'Ano Mes'] = df_expandido['Data fim']
df_expandido['Ano Mes'] = df_expandido['Ano Mes'].dt.to_period('M')
df_expandido['Data inicio'] = df_expandido['Data inicio'].dt.to_period('M')
df_expandido['Data fim'] = df_expandido['Data fim'].dt.to_period('M')

# elimina as linhas onde a operação não estava ativa
df_expandido = df_expandido[(df_expandido['Ano Mes'] >= df_expandido['Data inicio'])
                            & (df_expandido['Ano Mes'] <= df_expandido['Data fim'])]

# joga fora colunas desnecessárias para agilizar o código
colunas_a_manter = ['Ano Mes', 'Local de instalação', 'HH por mes']
colunas_a_manter.extend(colunas_tipo_atividade)
df_expandido = df_expandido[colunas_a_manter]

for coluna_tipo_atividade in colunas_tipo_atividade:
    # para cada categoria existente na coluna adiciona
    # uma nova coluna para guardar o HH daquela categoria
    for atividade in df_expandido[coluna_tipo_atividade].unique():
        if "HH de " + atividade in df_expandido.columns:
            df_expandido.loc[df_expandido[coluna_tipo_atividade] ==
                             atividade, "HH de " + atividade + " de " + coluna_tipo_atividade] = df_expandido["HH por mes"]
        else:
            df_expandido.loc[df_expandido[coluna_tipo_atividade] ==
                             atividade, "HH de " + atividade] = df_expandido["HH por mes"]

# limpa lixo consequente da maneira do agrupamento
df_agrupado = df_expandido.groupby(
    ['Ano Mes', 'Local de instalação']).sum().reset_index()
df_agrupado.drop(columns=colunas_tipo_atividade, inplace=True)
df_agrupado.rename({'HH por mes': 'HH total'}, inplace=True)
df_agrupado = df_agrupado[df_agrupado['Ano Mes'] >= '2020-01']

##### TRATAMENTO FINAL PARA PODER EXPORTAR A TABELA

In [ ]:
df_agrupado = df_agrupado.sort_values(
    by=["Ano Mes", "Local de instalação"]).reset_index(drop=True)

df_agrupado.fillna(0, inplace=True)

##### EXPORTANDO PLANILHA

In [ ]:
df_agrupado.to_csv(path_iw + "os_entrada.csv", index=False)